#내가 하는 태양광

In [ ]:
# 패키지 인포트!!!
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # CPU, GPU 선택해줌

import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

#pytorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#scikit-learn 라이브러리
from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

#pandas 라이브러리 임포트
import pandas as pd

#matplotlib 라이브러리 임포트
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline #이미지를 노트북 안에 출력하도록 하는것

UsageError: unrecognized arguments: #이미지를 노트북 안에 출력하도록 하는것


> 데이터 확인
* https://dacon.io/competitions/official/235683/data/
* 데이터 불러오기 이후 순차 실행
* 제일 핵심 요소만

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_raw = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/train/train.csv')
train= train_raw

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/sample_submission.csv')

In [ ]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96] # 왜 마지막 2일을 빼고 학습?

    elif is_train==False:
       
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :] # 왜 마지막 날들만 빼서 데이터를 모으지?


df_train = preprocess_data(train)
df_train.iloc[:48]

,Hour,TARGET,DHI,DNI,WS,RH,T,Target1,Target2
0,0,0.000000,0,0,1.5,69.08,-12,0.000000,0.000000
1,0,0.000000,0,0,1.5,69.06,-12,0.000000,0.000000
2,1,0.000000,0,0,1.6,71.78,-12,0.000000,0.000000
3,1,0.000000,0,0,1.6,71.75,-12,0.000000,0.000000
4,2,0.000000,0,0,1.6,75.20,-12,0.000000,0.000000
5,2,0.000000,0,0,1.5,69.29,-11,0.000000,0.000000
6,3,0.000000,0,0,1.5,72.56,-11,0.000000,0.000000
7,3,0.000000,0,0,1.4,72.55,-11,0.000000,0.000000
8,4,0.000000,0,0,1.3,74.62,-11,0.000000,0.000000
9,4,0.000000,0,0,1.3,74.61,-11,0.000000,0.000000


In [ ]:
train1= df_train
train1.iloc[:,1:8].min()
train1.iloc[:,1:8].max()

TARGET       99.913939
DHI         528.000000
DNI        1059.000000
WS           12.000000
RH          100.000000
T            35.000000
Target1      99.913939
dtype: float64

In [ ]:
# scaling 1
mini = train1.iloc[:,1:8].min()
size = train1.iloc[:,1:8].max() - train1.iloc[:,1:8].min()
train1.iloc[:,1:8] = (train1.iloc[:,1:8] -  mini) / size    # 민맥스 그 뭐냐 그 정규화임!!

input_window = 8       # 들어가는 수, 훈련 , 24시간 10일들어가서
output_window = 1       # 나오는 수, 테스트 또는 결과 , 24시간을 예측해봐

# 행렬을 먼저 만듬
window_x = np.zeros((train1.shape[0] - (input_window + output_window), input_window, 8))  # 데이터 전체에서 (들어가는 수랑 나오는 수)를 뺌(이게 for문 돌아갈때 마지막은 빼야하니까),  맨뒤에 4는 4개의 컬럼(사용자, 세션, 신규방문자, 페이지뷰),    (배치 사이즈, 시퀀스 길이, input 차원)
window_y = np.zeros((train1.shape[0] - (input_window + output_window), output_window, 8))


# 데이터 값을 넣음
for start in range(train1.shape[0] - (input_window + output_window)):  #데이터의 수만큼 돌아감, 하루씩 늘려가며 데이터 셋을 생성 다르게 말하면 데이터가 겹침!!!!
    end = start + input_window    # 데이터의 끝은 시작값과 들어가는 수, 훈련을 168행개
    window_x[start,:, :] = train1.iloc[start : end                , 1: ].values   # 한 데이터의 길이(배치 사이즈), 한데이터에 들어가는 데이터의 행 수(시퀸스 길이), 컬럼의 개수(인풋 차원) , 빈 행렬에 값을 넣기
    window_y[start,:, :] = train1.iloc[end   : end + output_window, 1: ].values   # 앞에 x에서 끝나는 뒤에, 24행개만 들어감, 그러니까 x랑 데이터가 겹칠 수 있으

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        
        self.hidden_lstm = nn.LSTM(input_size = input_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 8)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 8, hidden_size = 8)
model

LSTM(
  (lstm): LSTM(8, 8, batch_first=True)
  (hidden_lstm): LSTM(8, 8, batch_first=True)
  (time_fc): Linear(in_features=8, out_features=8, bias=True)
)

In [ ]:
window_x.shape

torch.Size([52455, 8, 8])

In [ ]:
# Model학습

window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)  # 1e-2
criterion = nn.MSELoss(size_average = True)
num_epochs  = 2000
train_error = []

for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 100 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

RuntimeError: ignored

# 성민이가 준 파일

In [ ]:
# 패키지 인포트!!!
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # CPU, GPU 선택해줌

In [ ]:
# 데이터 불러오기
train = pd.read_csv("../../1. 데이터/open_data/train.csv", encoding = 'euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train1  = train.groupby('date').sum().reset_index()  # 시간을 날짜별로 합쳐서 생성

In [ ]:
# 한번만!!!!!!!!!!!! 계속하면 0이됨
# 이거로 변경하자

# scaling
mini = train1.iloc[:,1:].min()
size = train1.iloc[:,1:].max() - train1.iloc[:,1:].min()
train1.iloc[:,1:] = (train1.iloc[:,1:] -  mini) / size    # 민맥스 그 뭐냐 그 정규화임!!

input_window = 10       # 들어가는 수, 훈련 , 24시간 10일들어가서
output_window = 1       # 나오는 수, 테스트 또는 결과 , 24시간을 예측해봐

# 행렬을 먼저 만듬
window_x = np.zeros((train1.shape[0] - (input_window + output_window), input_window, 4))  # 데이터 전체에서 (들어가는 수랑 나오는 수)를 뺌(이게 for문 돌아갈때 마지막은 빼야하니까),  맨뒤에 4는 4개의 컬럼(사용자, 세션, 신규방문자, 페이지뷰),    (배치 사이즈, 시퀀스 길이, input 차원)
window_y = np.zeros((train1.shape[0] - (input_window + output_window), output_window, 4))


# 데이터 값을 넣음
for start in range(train1.shape[0] - (input_window + output_window)):  #데이터의 수만큼 돌아감, 하루씩 늘려가며 데이터 셋을 생성 다르게 말하면 데이터가 겹침!!!!
    end = start + input_window    # 데이터의 끝은 시작값과 들어가는 수, 훈련을 168행개
    window_x[start,:, :] = train1.iloc[start : end                , 1: ].values   # 한 데이터의 길이(배치 사이즈), 한데이터에 들어가는 데이터의 행 수(시퀸스 길이), 컬럼의 개수(인풋 차원) , 빈 행렬에 값을 넣기
    window_y[start,:, :] = train1.iloc[end   : end + output_window, 1: ].values   # 앞에 x에서 끝나는 뒤에, 24행개만 들어감, 그러니까 x랑 데이터가 겹칠 수 있으

In [ ]:
# Model 생성
# 이걸로 가지고 놀꺼임

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-1:, :])
        
        return out_time.view(-1,1,4)

  
model = LSTM(input_size = 4, hidden_size = 15).to(device)  # 인풋은 들어가는 컬럼의 수를, 히든은 들어가는 중간 시퀸스 길이(날짜 혹은 들어가는 사이즈)

In [ ]:
# Model학습

window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 0.004)  # 1e-2
criterion = nn.MSELoss(size_average = True)
num_epochs  = 2000
train_error = []

for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 100 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

> 정리
* 딥러닝 시계열 계열을 돌릴때는 x와 y를 (데이터 수, 사용할 날짜, 컬럼 수)로 해야함!!
* 학습 시, 손실함수는 MSELoss